# Conceptos clave (visión profunda con CNN en CIFAR-10)

## Mapa rápido (8–10 líneas)
CIFAR-10 es un dataset de 60.000 imágenes RGB 32×32 organizadas en 10 clases (50.000 training, 10.000 test). Cada entrada tiene forma 32×32×3. Una CNN combina convoluciones (filtros locales) y pooling para extraer características espaciales jerárquicas; esto suele superar a una red totalmente densa porque las convoluciones explotan la invariancia por traslación (un objeto detectado en distintas posiciones sigue activando el mismo filtro) y comparten pesos espacialmente, reduciendo drásticamente parámetros frente a aplanar la imagen. Aplanar (flatten) al inicio ignora la estructura espacial y hace que la red necesite muchos más parámetros y datos para aprender patrones locales.

## Convolución sin magia
Un filtro (kernel) en Conv2D es una matriz de pesos de tamaño (k_h, k_w) que se aplica localmente sobre la imagen: por cada posición hace un producto elemento a elemento entre kernel y el parche de entrada y suma el resultado (más bias). Parámetros: tamaño del kernel (p. ej. 3×3), stride (paso entre posiciones), padding (valores añadidos en bordes: valid sin padding, same con padding para mantener tamaño), canales (kernel tiene dimensión adicional: in_channels y se producen out_channels filtros).

## Ejemplo:
Entrada 5×5, kernel 3×3, stride=1, padding=0 → salida 3×3.

Si entrada single-channel y kernel = [[1,0,-1],[1,0,-1],[1,0,-1]] (detector de bordes), convolucionas y obtienes activación que resalta transiciones horizontales.
Costo computacional aproximado por salida: k_h * k_w * in_channels multiplicaciones por cada ubicación y por cada filtro; total ≈ output_h * output_w * out_channels * k_h * k_w * in_channels.

## Pooling y por qué importa (Max vs Average, 6–8 líneas)
MaxPooling conserva la activación más fuerte en la ventana (preserva características discriminativas y es robusta a pequeñas traslaciones); AveragePooling promedia, suaviza y conserva información global pero puede atenuar rasgos fuertes. Max tiende a ayudar en clasificación porque mantiene las señales más relevantes; Average puede ayudar en tareas donde el conteo/energía promedio importa. Ambos reducen tamaño espacial y reducen sensibilidad al ruido local (al perder resolución). Ejemplo 2×2 (ventana): entrada [[1,2],[3,4]] → MaxPool → 4; AvgPool → (1+2+3+4)/4 = 2.5. Max aporta invarianza local mayor; Average es más informativa de la distribución local.

## Arquitectura típica de una CNN simple (texto)
Input 32×32×3 → [Conv(32,3×3) → ReLU → MaxPool(2×2)] × 2 → Flatten → Dense(128) → Dropout(0.5) → Dense(10, softmax)
Función de cada bloque (1 línea cada uno): Conv(32) extrae 32 mapas de características locales; ReLU introduce no-linealidad; Pool reduce resolución y hace invariancia local; Flatten transforma mapas en vector para capa densa; Dense(128) combina características globales; Dropout regulariza evitando co-adaptación; Dense(10, softmax) produce probabilidades por clase.

## Métrica y pérdida adecuadas

Pérdida estándar: Categorical Crossentropy (para etiquetas one-hot).

Métricas: accuracy (porcentaje de predicciones correctas) y, adicionalmente, precisión/recall por clase si interesa.
Accuracy mide aciertos globales pero oculta errores por clase; la matriz de confusión muestra qué clases se confunden y es clave en CIFAR-10 para detectar pares problemáticos (p. ej. cat vs dog).

# Profundizar en lo imprescindible

## Normalización y preparación de datos
Dividimos por 255.0 para llevar valores de píxeles de [0,255] a [0,1]; esto acelera convergencia y mantiene la magnitud de activaciones en rangos numéricamente estables. Estandarizar por canal (restar media por canal y dividir por desviación estándar del canal) es recomendable cuando la distribución por canal no es uniforme y cuando usas redes profundas o técnicas sensibles a escala (batchnorm ayuda también). Efectos: normalizar/estandarizar mejora estabilidad del gradiente, permite usar learning rates más grandes y reduce oscilaciones; sin normalización el entrenamiento puede ser lento o inestable.

## Baseline denso vs CNN (8–10 líneas)
Un MLP que aplana la imagen convierte cada píxel en una entrada independiente → necesita muchas conexiones para modelar relaciones espaciales locales (parámetros explotan con tamaño de entrada). Una CNN comparte pesos y limita conexiones a vecindad local (sesgo inductivo espacial): asume que patrones útiles son locales y repetibles. Resultado: la CNN tiene menos parámetros para la misma capacidad de detección de patrones locales, es más robusta a traslaciones y menos sensible al ruido de fondo; el MLP puede memorizar ruido y requiere mucho más dato para generalizar. En resumen: CNN = inductive bias útil para imágenes → mejora sample efficiency y generalización.

## Parámetros y capacidad (fórmula Conv2D)
Para una capa Conv2D con kernel k_h × k_w, in_c canales de entrada y out_c filtros (salida):
#parámetros = (k_h * k_w * in_c) * out_c + out_c (bias opcional).
Mayor kernel o más filtros aumentan capacidad (más parámetros), suben tiempo de entrenamiento y aumentan riesgo de overfit; profundidad (más capas) aumenta capacidad no lineal y coste computacional pero permite aprender jerarquías: mejor práctica es crecer primero en filtros por bloque y usar kernels pequeños (3×3) apilados.
## Regularización práctica (4 técnicas)
1. Dropout — apaga unidades aleatoriamente en entrenamiento; reduce co-adaptación; usar en capas densas y/o conv con bajo ratio (p. ej. 0.2–0.5).
2. L2 weight decay — penaliza pesos grandes; estabiliza y evita que la red memorize ruido.
3. Data Augmentation — genera más ejemplos via flips, pequeñas rotaciones, traslaciones, jitter de color; reduce overfitting.
4. Early Stopping — detener cuando la validación deja de mejorar; evita sobreentrenar al fin.
Combina augment + weight decay + dropout frecuentemente; early stopping actúa como seguridad final.

## Data Augmentation con cabeza (plan razonable para CIFAR-10)
- Flips horizontales: sí (70–100%).
- Rotaciones pequeñas: ±15° como máximo.
- Traslaciones: hasta ±4 píxeles (10–12% del tamaño).
- Jitter de color: brillo/contraste/saturación pequeños (±10–20%).
- NO aplicar flip vertical (cambia semántica para classes como 'truck' o 'frog' en algunos casos), NO rotaciones 90° si la orientación importa. Límites: mantener transformaciones realistas para las clases.

## Optimización y LR scheduling (SGD+momentum vs Adam, 6–8 líneas)
- SGD + momentum: convergencia estable, buen control de generalización, requiere ajuste cuidadoso de LR y schedule.
- Adam: adaptación automática del LR por parámetro, converge rápido y es robusto en inicialización; en algunas tareas generaliza ligeramente peor que SGD.
Scheduler recomendado: ReduceLROnPlateau (reduce LR cuando la validación se estanca) o CosineDecay para decay suave por época. Señales para bajar LR: valid loss que no mejora por N épocas, valid accuracy estancada o aumento de val loss.

## Curvas de aprendizaje: interpretación y patrones
- Subajuste: train loss alto, val loss alto y similares → aumentar capacidad (más filtros/profundidad), entrenar más, reducir regularización.
- Ajuste saludable: train loss baja y val loss baja, ambas mejorando → mantener, quizá entrenar más.
- Sobreajuste: train loss baja pero val loss sube/diverge → aumentar regularización, aumentar augment, usar early stopping o más datos.

## Matriz de confusión y clases difíciles
Leer la matriz: filas = etiquetas verdaderas, columnas = predicciones (o viceversa según convención). Pairs propensos: cat vs dog, truck vs automobile, bird vs plane (depende de backgrounds). Mejora: aumento específico (más ejemplos/augment para clases confundidas), añadir capacidad (más profundidad) o usar focal loss si clases desbalanceadas. -->

## Batch size y estabilidad
Batch size grande (128+) reduce ruido del gradiente y permite usar LR mayor pero puede generalizar peor; batch pequeño (32) añade ruido de gradiente que a veces actúa como regularizador. En Colab (GPU limitada) un valor inicial razonable: batch = 64 (balance entre estabilidad, uso de memoria y velocidad por época).

## Buenas prácticas de entrega (8–10 puntos)
1. Código limpio y reproducible (README).
2. Seed fijada (Python/Numpy/TF).
3. ENVIRONMENT.md y requirements.txt.
4. Logs de entrenamiento (tensorboard o CSV).
5. Curvas train/valid (loss y accuracy).
6. Matriz de confusión y análisis por clase.
7. Tabla comparativa Dense vs CNN (parámetros, accuracy, tiempos).
8. Resultados reproducibles (modelo final guardado).
9. Imágenes de ejemplo de aciertos/errores.
10. 5 hallazgos clave (qué funcionó, límite, overfitting, mejoras pendientes, tiempo de entrenamiento).

# Proyecto en Colab — PROMPT 1 (implementación práctica)

In [ ]:
# En Colab: instalar dependencias y montar repo (si quieres usar Drive)
!pip install -q tensorflow==2.14.0 scikit-learn pyyaml matplotlib



SyntaxError: invalid syntax (699319992.py, line 6)

In [8]:
# Pega esto en Colab (ajusta ruta a /content si subes carpeta)
import os, json, hashlib, datetime, random
import numpy as np
random.seed(42); np.random.seed(42)

from tensorflow.keras.datasets import cifar10
from sklearn.model_selection import StratifiedShuffleSplit
from tensorflow.keras.utils import to_categorical

(x_train_full, y_train_full), (x_test, y_test) = cifar10.load_data()
print("Formas:", x_train_full.shape, x_test.shape)

sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for tr, va in sss.split(x_train_full, y_train_full.ravel()):
    x_train, x_valid = x_train_full[tr], x_train_full[va]
    y_train, y_valid = y_train_full[tr], y_train_full[va]

# Normalizar y one-hot
x_train = x_train.astype('float32')/255.0
x_valid = x_valid.astype('float32')/255.0
x_test_norm = x_test.astype('float32')/255.0
y_train_oh = to_categorical(y_train, 10)
y_valid_oh = to_categorical(y_valid, 10)
y_test_oh  = to_categorical(y_test, 10)

# Hash primeras 1024 imágenes normalizadas
first_n = min(1024, x_train.shape[0])
subset = (x_train[:first_n]*255).astype('uint8')
h = hashlib.sha256(subset.tobytes()).hexdigest()

meta = {
  "x_train_shape": x_train.shape,
  "x_valid_shape": x_valid.shape,
  "x_test_shape": x_test_norm.shape,
  "created_at": datetime.datetime.utcnow().isoformat()+"Z",
  "sha256_first_{}_images".format(first_n): h
}
open('results/data_meta.json','w').write(json.dumps(meta, indent=2))
print("Guardado data_meta.json y shapes OK")


c:\Users\migue\Desktop\borrar\IA_P3_CIFAR10_Gonzalo_Millon\.venv\Lib\site-packages\keras\src\datasets\cifar.py:18: VisibleDeprecationWarning: dtype(): align should be passed as Python or NumPy boolean but got `align=0`. Did you mean to pass a tuple to create a subarray type? (Deprecated NumPy 2.4)
  d = cPickle.load(f, encoding="bytes")


Formas: (50000, 32, 32, 3) (10000, 32, 32, 3)
Guardado data_meta.json y shapes OK
